In [3]:
%pylab inline
# coding=utf-8

import tek3 # import tek3 as tek3 : utilidades de comunicación con Tek2002B
import time # import time
tds2002b = tek3.TekScope("/dev/usbtmc1") # creo una instancia de tek3.TekScope

# refresca lo importado de tek3 cada vez que se hace ejecución
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib
TEKTRONIX,TDS 2002B,C101891,CF:91.1CT FV:v22.11
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Funciones de lectura y archivado

In [4]:
def flName(camara, tipo, N):
    filePath= './data/'
    fileDate= time.strftime('%y%m%d')
    fileCamara= camara # 'n' nuestra = Campbelliana
    fileN= N
    fileExt= '.npy'
    fileType= tipo
    fileName= filePath+ fileType+ fileDate + fileCamara+ ('{:02n}'.format(fileN))+ fileExt
    return fileName


def escalas(osciloscopioTek):
    # obtener una vez los límites verticales del canal
    pos1 = osciloscopioTek.get_channel_position(1)
    scale1 = osciloscopioTek.get_channel_scale(1)
    pos2 = osciloscopioTek.get_channel_position(2)
    scale2 = osciloscopioTek.get_channel_scale(2)
    return pos1, scale1, pos2, scale2


def param(osciloscopioTek):
    '''
    Grabo parámetros de la medición
    '''
    osciloscopioTek.load_setup() # Obtiene parámetros del osciloscopio
    sets= osciloscopioTek.dico
    [pos1, scale1, pos2, scale2]= escalas(osciloscopioTek)
    sets['POSITION1']= pos1
    sets['SCALE1']= scale1
    sets['POSITION2']= pos2
    sets['SCALE2']= scale2
    sets['ELECTRONICA']= 'Luis'
    return sets

def readBuff2(self):
    try:
        return self.ask('CURVE?')
    except TimeoutError:
        print('Probably requested channel has no data')
        raise

def ptsAdq(osciloscopioTek):
    # correr una vez para obtener parámetros del buffer
    buff2 = readBuff2(osciloscopioTek) # esta bien la herencia?
    dataPointsCharacters= int(buff2[1:2])
    dataOffset= 2+ dataPointsCharacters
    return (int(buff2[2:2+ dataPointsCharacters])), dataOffset # número de lecturas en cada adquisición


def rapi(osciloscopioTek, dataCount, dataOffset):
    osciloscopioTek.start_acq()
    while(osciloscopioTek.textAsk('BUSY?')=='1'):
        time.sleep(1E-3)
    buff2 = readBuff2(osciloscopioTek) # esta bien la herencia?
    return ( np.frombuffer(buff2, dtype = np.dtype('int8').newbyteorder('<'), count= dataCount, offset= dataOffset) )


def lectura(osciloscopioTek, adquisiciones, puntosAdquisicion, dataOffset, tiempoInicial):
    # la lectura en sí
    ch1= np.empty([adquisiciones,puntosAdquisicion], dtype ='int8')
    ch2= np.empty([adquisiciones,puntosAdquisicion], dtype ='int8')
    zeit= tek3.TekScope.Xaxis(osciloscopioTek) # zeit
    for medicion in range(adquisiciones):
        print('\rmedición '+ str(medicion)+ ' de '+ str(adquisiciones), end=' ')
        osciloscopioTek.set_data_source(1) # canal 2
        ch1[medicion]= rapi(osciloscopioTek, puntosAdquisicion, dataOffset)
        osciloscopioTek.set_data_source(2) # canal 2
        ch2[medicion]= rapi(osciloscopioTek, puntosAdquisicion, dataOffset)
    print('\rmedición '+ str(medicion+ 1)+ ' de '+ str(adquisiciones))
    print('Tiempo transcurrido= {:.1f} min'.format((time.time()- tiempoInicial)/60) ) # tiempo transcurrido en segundos
    return zeit, ch1, ch2


def adquiere(osciloscopioTek, medicionDia, settings, adquisiciones, descripcionDetector, descripcionArchivo, puntosAdquisicion, dataOffset, trgLevel= -0.058):
    '''
    registra ambos canales en formato de multiples arrays
    '''
    tiempoInicial= time.time()
    # ch2 =lectura(adquisiciones, puntosAdquisicion, dataOffset, tiempoInicial)
    [zeit, ch1, ch2]= lectura(osciloscopioTek, adquisiciones, puntosAdquisicion, dataOffset, tiempoInicial)
    # PULSER INVIERTO SIGNO CH1: Mantengo el ánalisis imperturbado
    ch1*= -1
    # graba    
    np.savez_compressed(flName(descripcionDetector, descripcionArchivo, medicionDia), settings=settings, zeit=zeit, ch1=ch1, ch2=ch2)

# Configuración previa a primer adquisición

In [5]:
# Secuencia principal
tds2002b.write('INITIATE_CLEAR')
tds2002b.write('HEADER OFF')
tds2002b.set_data_encoding('RIBinary')
tds2002b.write('ACQuire:MODe SAMple') # test.write('ACQuire:MODe PEAKdetect') lo descarto pues manual dice que puede verse mas ruidoso  
tds2002b.write('ACQUIRE:STOPAFTER SEQuence')
tds2002b.write('ACQuire:STATE RUN')
tds2002b.write('CH1:COUPling DC')
tds2002b.write('CH2:COUPling DC')

In [6]:
# Cámaras
tds2002b.write('CH1:SCAle 0.05')  # FC-165 180226
# tds2002b.write('CH1:SCAle 0.02')  # FC-165 171220
# tds2002b.write('CH1:SCAle 0.01')

tds2002b.write('CH1:POSition -1.04')  # FC-165 171220
# tds2002b.write('CH1:POSition -2.0')

tds2002b.write('CH2:SCAle 1.0') # FC-165 180226
# tds2002b.write('CH2:SCAle 0.5') # FC-165 171220
# tds2002b.write('CH2:SCAle 1.0') # CFP-035 161007
# tds2002b.write('CH2:SCAle 0.5') # Ensayo DC-045 160926

tds2002b.write('CH2:POSition -3.04') # FC-165 171220
# tds2002b.write('CH2:POSition -2.00')

tds2002b.horizontal_main_scale(1.0E-7)  # FC-165 171220
# tds2002b.horizontal_main_scale(2.5E-7)  # Ensayo DC-045 160926

# tds2002b.horizontal_main_position(2.0E-7) # FC-165 171220
# tds2002b.horizontal_main_position(2.0E-8) # 2.5E-7 para pul, lo ensayo para acq
# tds2002b.horizontal_main_position(1.0E-6)  # Ensayo DC-045 160926
# tds2002b.horizontal_main_position(5.0E-7)  # Ensayo CFP-035 161007

tds2002b.trigger_source('CH2')
tds2002b.trigger_level(1.2) # [V] FC-165 180226
# tds2002b.trigger_level(0.26) # [V] ensayo FC-165 171220
# tds2002b.trigger_level(1.96) # 1.96 [V] nominal para CFP-35 con electrónica modernizada 151016
# tds2002b.trigger_level(1.4) # Ensayo DC-045 160926
#tds2002b.trigger_level(1.0) # 1.96 [V] FC-165 161013

In [7]:
# generador pulsos
tds2002b.write('CH1:SCAle 0.02')
tds2002b.write('CH1:POSition 3.0')
tds2002b.write('CH2:SCAle 0.01')
tds2002b.write('CH2:POSition 0.0')
tds2002b.horizontal_main_scale(1.0E-7)
tds2002b.horizontal_main_position(3.16E-7)
tds2002b.trigger_source('CH1')
tds2002b.trigger_level(-0.04) # [V]

In [8]:
settings= param(tds2002b)
[puntosAdquisicion, dataOffset] = ptsAdq(tds2002b) # puntos por cada adquisición, headbits en cada adquisición

# Secuencia a ejecutar en cada adquisición

In [6]:
descripcionDetector= 'z'
descripcionArchivo= 'pul'
# descripcionArchivo= 'acq'
adquisiciones= 1*1280 # número de lecturas

for medicionDia in np.arange(1,2): # medición 1 a 2 (dos mediciones)
    adquiere(tds2002b, medicionDia, settings, adquisiciones, descripcionDetector, descripcionArchivo, puntosAdquisicion, dataOffset)

medición 1280 de 1280
Tiempo transcurrido= 28.4 min


# Ensayos

In [6]:
# tds2002b.horizontal_main_position(0)
[ch1S, ch1p, ch2s, ch2p, hS, hP, trig]= [tds2002b.get_channel_scale('CH1'), tds2002b.get_channel_position('CH1'), tds2002b.get_channel_scale('CH2'), tds2002b.get_channel_position('CH2'), tds2002b.horizontal_main_scale(), tds2002b.horizontal_main_position(), tds2002b.trigger_level()]
[ch1S, ch1p, ch2s, ch2p, hS, hP, trig]

[0.02, -1.04, 0.5, -3.04, '1.0E-7', '2.0E-7', 0.26]

In [2]:
tds2002b.textAsk('BUSY?')=='1' 

False

In [6]:
settings= param(tds2002b)
[puntosAdquisicion, dataOffset] = ptsAdq(tds2002b) # puntos por cada adquisición, headbits en cada adquisición
settings, puntosAdquisicion, dataOffset

({':ACQUIRE:MODE': 'SAMPLE',
  ':AUTORANGE:SETTINGS': 'BOTH',
  ':CH1:PROBE': '1.0E0',
  ':CH2:PROBE': '1.0E0',
  ':CURSOR:FUNCTION': 'VBARS',
  ':CURSOR:HBARS:POSITION1': '6.0E-1',
  ':CURSOR:VBARS:UNITS': 'SECONDS',
  ':DATA:ENCDG': 'RIBINARY',
  ':DISPLAY:FORMAT': 'YT',
  ':HARDCOPY:BUTTON': 'SAVESALL',
  ':HORIZONTAL:DELAY:SCALE': '5.0E-9',
  ':HORIZONTAL:VIEW': 'MAIN',
  ':LANGUAGE': 'SPANISH',
  ':LOCK': 'NONE',
  ':MATH:DEFINE': '"CH1-CH2"',
  ':MATH:FFT:HORIZONTAL:POSITION': '5.0E1',
  ':MATH:FFT:VERTICAL:POSITION': '0.0E0',
  ':MEASUREMENT:IMMED:TYPE': 'PERIOD',
  ':MEASUREMENT:MEAS1:TYPE': 'CRMS',
  ':MEASUREMENT:MEAS2:TYPE': 'FREQUENCY',
  ':MEASUREMENT:MEAS3:TYPE': 'NWIDTH',
  ':MEASUREMENT:MEAS4:TYPE': 'PK2PK',
  ':MEASUREMENT:MEAS5:TYPE': 'FREQUENCY',
  ':MEASUREMENT:METHOD': 'AUTO',
  ':PICTBRIDGE:PAPERSIZE': 'DEFLT',
  ':SAVE:IMAGE:FILEFORMAT': 'EPSIMAGE',
  ':SELECT:CH1': '1',
  ':TRIGGER:MAIN:EDGE:SOURCE': 'CH1',
  ':TRIGGER:MAIN:LEVEL': '-1.12E-3',
  ':TRIGGER:MAIN:M

In [4]:
l = tds2002b.textAsk('SET?')

In [5]:
lok= [e.split(' ') for e in l.split(';')[1:]]

In [6]:
lok

[[':VERBOSE', '1'],
 [':DATA:ENCDG', 'RIBINARY'],
 ['DESTINATION', 'REFA'],
 ['SOURCE', 'CH2'],
 ['START', '1'],
 ['STOP', '2500'],
 ['WIDTH', '1'],
 [':LOCK', 'NONE'],
 [':DISPLAY:FORMAT', 'YT'],
 ['STYLE', 'VECTORS'],
 ['PERSISTENCE', '0'],
 ['CONTRAST', '54'],
 ['INVERT', 'OFF'],
 [':ACQUIRE:MODE', 'SAMPLE'],
 ['NUMAVG', '64'],
 ['STATE', '1'],
 ['STOPAFTER', 'RUNSTOP'],
 [':CH1:PROBE', '1.0E0'],
 ['CURRENTPROBE', '1.0E1'],
 ['SCALE', '2.0E-3'],
 ['POSITION', '5.2E-1'],
 ['COUPLING', 'AC'],
 ['BANDWIDTH', 'OFF'],
 ['INVERT', 'OFF'],
 ['YUNIT', '"V"'],
 [':CH2:PROBE', '1.0E0'],
 ['CURRENTPROBE', '1.0E1'],
 ['SCALE', '5.0E-1'],
 ['POSITION', '-3.16E0'],
 ['COUPLING', 'AC'],
 ['BANDWIDTH', 'OFF'],
 ['INVERT', 'OFF'],
 ['YUNIT', '"V"'],
 [':HORIZONTAL:VIEW', 'MAIN'],
 ['MAIN:SCALE', '5.0E-6'],
 ['POSITION', '1.08E-7'],
 [':HORIZONTAL:DELAY:SCALE', '5.0E-9'],
 ['POSITION', '3.2E-8'],
 [':TRIGGER:MAIN:MODE', 'AUTO'],
 ['TYPE', 'EDGE'],
 ['HOLDOFF:VALUE', '5.0E-7'],
 [':TRIGGER:MAIN:EDGE:S